In [ ]:
# tradingbot.py
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from cons_range_indicator import ConsolidationRangeDetector
from smc_indicator_v6 import SMCStructuresFVG
from config2 import TERMINAL_PATH, ACCOUNT, PASSWORD, SERVER
import os
from discord_report import DiscordReporter


discord_webhook = "https://discord.com/api/webhooks/…"
reporter = DiscordReporter(discord_webhook)


# Initialize MT5
print("Initializing MetaTrader 5...")
if not mt5.initialize(
    path=TERMINAL_PATH,
    login=ACCOUNT,
    password=PASSWORD,
    server=SERVER
):
    raise RuntimeError("MT5 initialization failed")

# Map timeframe names to MT5 constants
TIMEFRAMES = {
    # '1D': mt5.TIMEFRAME_D1,
    # '4h': mt5.TIMEFRAME_H4,
    '1h': mt5.TIMEFRAME_H1,
    '30m': mt5.TIMEFRAME_M30,
    # '5m': mt5.TIMEFRAME_M5,
    #'1m': mt5.TIMEFRAME_M1,
}

# Fetch OHLCV data
def fetch_ohlcv(symbol: str, timeframe: int, n: int) -> pd.DataFrame:
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to fetch {symbol} data for timeframe {timeframe}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df[['time', 'open', 'high', 'low', 'close', 'tick_volume']]
    df.rename(columns={'tick_volume': 'volume'}, inplace=True)
    return df

# Main process
def main():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 0)
    pd.set_option('display.max_rows', None)

    symbol = 'BTCUSDc'
    n_bars = 100

    for name, tf in TIMEFRAMES.items():
        print(f"\n--- Processing timeframe: {name} ---")
        df = fetch_ohlcv(symbol, tf, n_bars)
        print(f"Fetched {len(df)} rows for {name}")
        
        


        smc = SMCStructuresFVG(df)
        result_df, confluence_df = smc.run()

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        audit_file = f"smc_audit_{name}_{timestamp}.csv"
        conf_file = f"smc_confluence_{name}_{timestamp}.csv"

        result_df.to_csv(audit_file, index=False)
        confluence_df.to_csv(conf_file, index=False)

        print(f"Saved audit: {audit_file}")
        print(f"Saved confluence: {conf_file}")

    mt5.shutdown()
    print("MT5 shutdown")

if __name__ == '__main__':
    main()


 MT5 terminal exists: True
 Python interpreter: 64bit
 ✅ MT5 connected: Login=183342154, Balance=6496.31, FreeMargin=2838.3
Initializing MetaTrader 5...

--- Processing timeframe: 1h ---
Fetched 100 rows for 1h
Saved audit: smc_audit_1h_20250710_090322.csv
Saved confluence: smc_confluence_1h_20250710_090322.csv

--- Processing timeframe: 30m ---
Fetched 100 rows for 30m
Saved audit: smc_audit_30m_20250710_090323.csv
Saved confluence: smc_confluence_30m_20250710_090323.csv
MT5 shutdown
